In [1]:
#EdiPRompt framework and all its meta-prompts
#Note: we run ediprompt stages indiovudally to ensure they are working
#Can easily be made automatic for future research if ran on hundreds of iterations
#But as only doing 8 iterations wanted to ensure each stage was working
#This is the code once it was completed, hence why we have 5 instruciton-score pairs for inital and mutated

In [2]:
import numpy as np
import pandas as pd
import json
import os
import openai

openai_api_key = ''

df_train = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/trainwindowsimproved.csv")
df_test = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/testwindowsimproved.csv")


In [3]:
def ask_gpt(question):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question}
            ]
        )

        answer = response['choices'][0]['message']['content']
        tokens_used = response['usage']['total_tokens']
        input_tokens = len(question.split()) 
        return answer, tokens_used, input_tokens

    except openai.error.OpenAIError as e:
        return f"Error: {str(e)}", 0, 0
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}", 0, 0

In [6]:
def get_template_intial_instruction(examples_df):
    
    metaPrompt = """
Your task is to generate the instruction <INS>. Below are some previous instructions with their scores. The score is the f-1 score and ranges from 0 to 1. We want to maximise this score.
    
Instruction: Review the conversation carefully for any signs that the individual falls into our vulnerability categories: significant Health issues impacting their everyday tasks, major Life events, low Resilience, or poor Capability in managing finances and other critical activities. If you detect factors that meet these conditions of vulnerability, answer plainly with 'Yes'. If no such factors are present, answer directly with 'No'. Provide only a 'Yes' or 'No' response, grounded strictly in the observable evidence of vulnerability from the dialogue.
Score: 0.662

Instrution: Examine the dialogue attentively for unequivocal evidence of customer vulnerability, as per our criteria: severe health conditions hampering everyday living, pivotal life changes (e.g., loss of a loved one or employment), diminished resilience to cope with setbacks, or challenges in managing finances or fundamental duties. Issue a definitive 'Yes’ only when these vulnerability characteristics are clearly observed in the conversation, or a 'No' if they are not. Your response should be strictly binary: 'Yes' or 'No'.
Score: 0.675

Instruction: Determine the customer's vulnerability from the interaction by identifying any explicit references that match our vulnerability criteria: substantial daily life impairment due to health conditions; critical life events such as bereavement, unemployment, or relationship breakdown; diminished ability to cope with unforeseen financial or emotional challenges; or evident difficulties in managing fiscal responsibilities and essential tasks. If any of these criteria are distinctly mentioned, respond decisively with 'Yes'. Otherwise, provide a clear 'No'. Your answer must be strictly a singular 'Yes' or 'No', based on the observable evidence within the dialogue. Answer: <Yes/No>
Score: 0.687

Instruction: Scrutinize the exchange to detect any clear evidence of the customer being at risk due to serious health conditions, impactful life changes, weakened resilience, or impaired ability in crucial domains like handling finances. Respond exclusively with 'Yes' if indicators of vulnerability are identified, or 'No' if they are absent, according strictly to the exchange, and limit your answer strictly to 'Yes' or 'No'.
Score: 0.705

Instruction: Analyze the dialogue meticulously for undeniable signs that the customer may be vulnerable, in line with our criteria: significant health issues that impede daily routines, profound life events, resilience in the face of hardships, or capability in financial management and other pivotal areas. Provide a succinct 'Yes' if such signs are evident, or a 'No' otherwise, drawing strictly from the concrete evidence in the conversation. Your answer must solely consist of 'Yes' or 'No'.
Score: 0.714

    
Below are examples of the problems, we are classifying customers conversations as vulnerable or not vulnerable. Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers:
Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.
Life events - life events such as bereavement, job loss or relationship breakdown.
Resilience - low ability to withstand financial or emotional shocks.
Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
"""
    
    #INSERTING EXAMPLES OF THE PROBLEM TO LET GPT-4 lEARN THE DOMAIN
    example_string = ""
    for _, example_row in examples_df.iterrows():
        formatted_utterance = example_row['conversation_chunks']
        #example_string += "\n" + formatted_utterance + "\n" 
        ins_text = "<INS>"
        example_string += "\n" + "Conversation: " + formatted_utterance + "\n" + ins_text + "\n" + "Answer: " + ("Yes" if example_row['labels'] == 1 else "No") + "\n"

    instruction = """\nGenerate an instruction that is different from all the instructions above, and has a higher score than all the instructions above. The instruction should be concise, effective, and generally applicable to the classifcation task at hand. The instruction must contain that we need a Yes or No response.\n"""


    prompt = metaPrompt + example_string + instruction
    
    return prompt

In [7]:
def get_template_sem_similar():
    
    instruction = """\nHere is the current instruction: Scrutinize the dialogue for indicators of vulnerability according to the following distinct criteria: health issues hindering day-to-day life, significant life disruptions such as loss, unemployment, or family difficulties, frailty in dealing with unexpected personal or financial crises, and limited understanding or self-assurance in money handling or other essential competencies. If the discourse unmistakably reflects these vulnerability factors, respond with a definitive 'Yes'. If these attributes are not present, issue a 'No'. Your response must be a categorical 'Yes' or 'No', based solely on the evidence in the exchange.\n
Generate two new instructions that are semantically similar to the instruction above. The instruction should be concise, effective, and generally applicable to the classifcation task at hand. The instruction must contain that we need a Yes or No response.\n"""

    prompt = instruction
    return prompt

In [14]:
def get_template_crossover():
    
    instruction = """Below are two instructions with their scores, the score is the f-1 score. We want to maximize this score.\n

Instruction 1: "Scrutinize the dialogue for indicators of vulnerability according to the following distinct criteria: health issues hindering day-to-day life, significant life disruptions such as loss, unemployment, or family difficulties, frailty in dealing with unexpected personal or financial crises, and limited understanding or self-assurance in money handling or other essential competencies. If the discourse unmistakably reflects these vulnerability factors, respond with a definitive 'Yes'. If these attributes are not present, issue a 'No'. Your response must be a categorical 'Yes' or 'No', based solely on the evidence in the exchange."  
Score: 0.694

Instruction 2: "Analyze the dialogue meticulously for undeniable signs that the customer may be vulnerable, in line with our criteria: significant health issues that impede daily routines, profound life events, resilience in the face of hardships, or capability in financial management and other pivotal areas. Provide a succinct 'Yes' if such signs are evident, or a 'No' otherwise, drawing strictly from the concrete evidence in the conversation. Your answer must solely consist of 'Yes' or 'No'."
Score: 0.714

Your task is to cross-over the above two instructions, combining the most effective elements from each to create a new, improved instruction. Also provide an explination into the crossover and what elements you have taken from each instruction.\n"""

    prompt = instruction
    

    #print(prompt)
    return prompt

In [17]:
def get_template_mutation(examples_df):
    
    metaPrompt = """
    Your task is to mutate the current crossover instruction to create a new instruction <INS>. Below are some previous instructions with their scores, the score is the f-1 score. We want to maximize this score.    
    
    Instruction: Review the conversation carefully for any signs that the individual falls into our vulnerability categories: significant Health issues impacting their everyday tasks, major Life events, low Resilience, or poor Capability in managing finances and other critical activities. If you detect factors that meet these conditions of vulnerability, answer plainly with 'Yes'. If no such factors are present, answer directly with 'No'. Provide only a 'Yes' or 'No' response, grounded strictly in the observable evidence of vulnerability from the dialogue.
    Score: 0.662

    Instruction: Determine the customer's vulnerability from the interaction by identifying any explicit references that match our vulnerability criteria: substantial daily life impairment due to health conditions; critical life events such as bereavement, unemployment, or relationship breakdown; diminished ability to cope with unforeseen financial or emotional challenges; or evident difficulties in managing fiscal responsibilities and essential tasks. If any of these criteria are distinctly mentioned, respond decisively with 'Yes'. Otherwise, provide a clear 'No'. Your answer must be strictly a singular 'Yes' or 'No', based on the observable evidence within the dialogue. Answer: <Yes/No>
    Score: 0.687

    Instrution: Examine the dialogue attentively for unequivocal evidence of customer vulnerability, as per our criteria: severe health conditions hampering everyday living, pivotal life changes (e.g., loss of a loved one or employment), diminished resilience to cope with setbacks, or challenges in managing finances or fundamental duties. Issue a definitive 'Yes’ only when these vulnerability characteristics are clearly observed in the conversation, or a 'No' if they are not. Your response should be strictly binary: 'Yes' or 'No'.
    Score: 0.675

    Instruction: Scrutinize the exchange to detect any clear evidence of the customer being at risk due to serious health conditions, impactful life changes, weakened resilience, or impaired ability in crucial domains like handling finances. Respond exclusively with 'Yes' if indicators of vulnerability are identified, or 'No' if they are absent, according strictly to the exchange, and limit your answer strictly to 'Yes' or 'No.'
    Score: 0.705

    Instruction: Analyze the dialogue meticulously for undeniable signs that the customer may be vulnerable, in line with our criteria: significant health issues that impede daily routines, profound life events, resilience in the face of hardships, or capability in financial management and other pivotal areas. Provide a succinct 'Yes' if such signs are evident, or a 'No' otherwise, drawing strictly from the concrete evidence in the conversation. Your answer must solely consist of 'Yes' or 'No'.
    Score: 0.714

    

    Below are examples of the problems, we are classifying customers conversations as vulnerable or not vulnerable. Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers:
Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.
Life events - life events such as bereavement, job loss or relationship breakdown.
Resilience - low ability to withstand financial or emotional shocks.
Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
"""

    #INSERTING EXAMPLES OF THE PROBLEM TO LET GPT-4 lEARN THE DOMAIN
    example_string = ""
    for _, example_row in examples_df.iterrows():
        formatted_utterance = example_row['conversation_chunks'] 
        ins_text = "<INS>"

        example_string += "\n" + "Conversation: " + formatted_utterance + "\n" + ins_text + "\n" + "Answer: " + ("Yes" if example_row['labels'] == 1 else "No") + "\n"

   
    instruction = """\nCrossover Instruction: Carefully examine the dialogue for unequivocal signs of vulnerability, employing the specified benchmarks: health problems that substantially interfere with daily activities, major life upheavals like bereavement, joblessness, or familial issues, resilience or fragility when faced with unexpected personal or financial troubles, and adequacy in managing finances or other crucial life skills. If you discern these vulnerability indicators clearly evidenced in the conversation, state a resolute 'Yes'. Otherwise, provide a 'No'. Render your verdict as either 'Yes' or 'No', based strictly on tangible proof found in the dialogue.\n
    Your task is to mutate the above crossover instruction, to generate a new improved instruction that is different from all the instructions above, and has a higher score than all the instructions above. The instruction should be concise, effective, and generally applicable to the classification task at hand. The instruction must contain that we need a Yes or No response."""
    prompt = metaPrompt + example_string + instruction
    

    print(prompt)
    return prompt

In [18]:
def query_api_ins(num_examples):
    # Initialize an empty list to store the results
    results = []
    total_input_tokens = 0
    total_output_tokens = 0
    
    
    #prompt = get_template_intial_instruction(df_train.sample(num_examples))
    #prompt = get_template_sem_similar()
    #prompt = get_template_crossover()
    prompt = get_template_mutation(df_train.sample(num_examples))
    print(prompt)
        
    answer, tokens_used, input_tokens = ask_gpt(prompt)
    total_input_tokens += input_tokens
    total_output_tokens += tokens_used - input_tokens

        
  
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    return answer


In [ ]:
query_api_ins(12)

In [4]:
#TESTING GENRATED PROMPTS IN ZERO-SHOT ENVIRONMENT
def get_template_test(example):
    
    metaPrompt = """Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers.
    Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.

    Life events - life events such as bereavement, job loss or relationship breakdown.

    Resilience - low ability to withstand financial or emotional shocks.

    Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
    """

    conversation = "\n" + "Conversation: " + example + "\n"

    #This changes for every new instruction we test
    instruction = "\nLet's think step by step, Rigorously scrutinize the dialogue for definitive indications of the customer's vulnerability using our established criteria: significant health complications affecting day-to-day functionality, severe life events like loss, unemployment, or domestic disturbances, lack of resilience to socioeconomic or emotional adversity, and proficiency in handling monetary affairs or essential life competencies. Confirm a clear 'Yes' if such vulnerability attributes are concretely presented in the discussion, or state 'No' in their absence. Your determination should be a clear-cut 'Yes' or 'No', anchored in the solid evidence of vulnerability disclosed through the interaction. Your answer must be strictly 'Yes' or 'No'.\n"
    prompt = metaPrompt + conversation + instruction

    return prompt

In [8]:
def query_api_test(sampled_df):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    
    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']
        prompt = get_template_test(utterance)
        #print(prompt)
        
        
        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

       
        answer = answer.rstrip('.').strip()

        
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        #print(f"Utterance: {utterance}")
        #print(f"Actual Label: {row['labels']}")
        #print(f"Answer: {answer}")
        # Store the example, actual label, and chatGPT label in the results list
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    # Convert the results list into a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
results = query_api_test(df_test)

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


actual_labels = results['Actual Label'].astype(int) 
predicted_labels = results['ChatGPT Label'].astype(int)


accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

confusion = confusion_matrix(actual_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)
